# Design Patterns in CZ2006

### Questions to Keep in Mind

1. When should I apply a design pattern?
2. What problem is the pattern trying to solve?
3. How does the design pattern work?
4. What are the benefits and costs of applying the design pattern?

In [2]:
from abc import ABC, abstractmethod
import random

## Strategy Pattern

The Strategy Pattern defines a family of algorithms, encapsulates each one, and makes them interchangeable. Strategy lets the algorithm vary independently from the clients that use it.

**It enables selecting an algorithm at runtime.**

![](uml-diagrams/strategy-pattern.png)

Note to self: Focus on the has-a relationship between Context and AbstractStrategy. It's kind of like letting a class have a behavior as its attribute. E.g. Context could have a ContextStrategyA algorithm as its attribute. When it wants to do carry out its business logic utilizing the algorithm, it simply calls performStrategy. It doesn't know how to perform the algorithm - that's for AbstractStrategy to take care of.

#### Pros of Strategy Pattern

1. Encapsulation
2. Implementation hiding
3. Ability to change behavior at runtime

#### Cons of Strategy Pattern

1. Complex, hard-to-understand code

### Skelly Code

In [12]:
"""
The class that the client is interested in.
"""
class Context:
    # Pass a Strategy object into the constructor
    def __init__(self, strategy):
        self.strategy = strategy
    
    # Allows the replacement of a Strategy object at runtime
    def setStrategy(self, strategy):
        self.strategy = strategy
    
    # Instead of implementing multiple versions of doSomething on its own,
    # The work is delegated to the Strategy object
    def doBusinessLogic(self):
        self.strategy.doSomething()

"""
Interface declaring operations common to all versions of some algorithm.
Context uses this interface to call the algorithm defined by Concrete Strategies.
Context doesn't know how Strategy will do it, but it knows it'll get done.
"""    
class AbstractStrategy(ABC):
    @abstractmethod
    def doSomething(self):
        pass

"""
Implementations of AbstractStrategy.
"""
class ConcreteStrategyA(AbstractStrategy):
    def doSomething(self):
        print("I'm ConcreteStrategyA!")
        
class ConcreteStrategyB(AbstractStrategy):
    def doSomething(self):
        print("I'm ConcreteStrategyB!")
        
c = Context(ConcreteStrategyA())
c.doBusinessLogic()

print("---")

c.setStrategy(ConcreteStrategyB())
c.doBusinessLogic()

I'm ConcreteStrategyA!
---
I'm ConcreteStrategyB!


### Example 1: Duckies

In [8]:
class FlyBehavior(ABC):
    def fly(self):
        pass
    
class FlyWithWings(FlyBehavior):
    def fly(self):
        print("I'm flying!")
        
class FlyNoWay(FlyBehavior):
    def fly(self):
        print("I can't fly. :(");

In [9]:
class QuackBehavior(ABC):
    @abstractmethod
    def quack(self):
        pass
    
class Quack(QuackBehavior):
    def quack(self):
        print("Quack!");
        
class MuteQuack(QuackBehavior):
    def quack(self):
        print("...")
        
class Squeak(QuackBehavior):
    def quack(self):
        print("Squeak~")

In [10]:
class Duck(ABC):
    @abstractmethod
    def display(self):
        pass
    
    def performFly(self):
        self.flyBehavior.fly()
    
    def performQuack(self):
        self.quackBehavior.quack()
        
    def swim(self):
        print("All ducks float, even decoys!")
    
    # Methods to change the behavior of a duck on the fly
    def setFlyBehavior(self, fb):
        self.flyBehavior = fb
        
    def setQuackBehavior(self, qb):
        self.quackBehavior = qb

class MallardDuck(Duck):
    def __init__(self):
        self.quackBehavior = Quack()
        self.flyBehavior = FlyWithWings()
    
    def display(self):
        print("I'm a mallard duck! uwu")

In [11]:
mallard = MallardDuck()
mallard.performQuack()
mallard.performFly()

print("---")

mallard.setFlyBehavior(FlyNoWay())
mallard.performFly()

Quack!
I'm flying!
---
I can't fly. :(


## Observer Pattern

The Observer Pattern defines a one-to-many dependency between objects so that when one object changes state, all of its dependents are notified and updated automatically.

![](uml-diagrams/observer-pattern.png)

#### Pull Model (2-Way Communication)

- When the subject's state changes, it notifies its observers **that its state has changed**
- If the observer is interested, it can call back for details by calling subject.getData()


#### Push Model (1-Way Communication)

- When the subject's state changes, it notifies its observers **by sending detailed information about the update**, whether or not the Observer is interested or not
- Hence, the observer never needs to call back



#### Pros of Observer Pattern

1. Loose coupling between a Subject and its Observers
    - Subjects will notify their Observers once a change occurs
    - Observers are allowed to change their interests in a Subject freely
    - Observers want to know a Subject's change when it occurs but don't know when it will happen
2. Reusability of Subjects and Observers as they are independent of each other
3. Broadcast communication

#### Cons of Observer Pattern

1. Slower performance
2. Higher complexity

### Skelly Code

In [36]:
class AbstractSubject(ABC):
    @abstractmethod
    def register(self, observer):
        pass
    
    @abstractmethod
    def unregister(self, observer):
        pass
    
    @abstractmethod
    def notify(self):
        pass
    
class AbstractObserver(ABC):
    @abstractmethod
    def update(self, subject):
        pass

### Pull Model

In [38]:
class ConcreteSubject(AbstractSubject):
    state = None
    observers = []
    
    def register(self, observer):
        self.observers.append(observer)
        
    def unregister(self, observer):
        self.observers.remove(observer)
    
    """Subject passed as parameter"""
    def notify(self):
        for observer in self.observers:
            observer.update(self)
            
    def getState(self):
        return self.state
            
    def doBusinessLogic(self):
        self.state = random.randint(0, 10)
        self.notify() # Notify when state changes
        
class ConcreteObserverA(AbstractObserver):
    """Observer calls Subject's getters"""
    def update(self, subject):        
        print(f"> ConcreteObserverA was notified of Subject's new state: {subject.getState()}")
    
class ConcreteObserverB(AbstractObserver):
    def update(self, subject):
        print(f"> ConcreteObserverB was notified of Subject's new state: {subject.getState()}")
            
subject = ConcreteSubject()

# Register new observers
observerA = ConcreteObserverA()
subject.register(observerA)
observerB = ConcreteObserverB()
subject.register(observerB)

subject.doBusinessLogic()

print("---")

subject.unregister(observerA)
subject.doBusinessLogic()

> ConcreteObserverA was notified of Subject's new state: 4
> ConcreteObserverB was notified of Subject's new state: 4
---
> ConcreteObserverB was notified of Subject's new state: 8


### Push Model

Focus on the difference in the subject's **notify() method**, and that the observers' **update() methods** take in the subject's state - not the subject object itself.

In [37]:
class ConcreteSubject(AbstractSubject):
    state = None
    observers = []
    
    def register(self, observer):
        self.observers.append(observer)
        
    def unregister(self, observer):
        self.observers.remove(observer)

    """State passed in instead of subject"""
    def notify(self):
        for observer in self.observers:
            observer.update(self.state) 
            
    def doBusinessLogic(self):
        self.state = random.randint(0, 10)
        self.notify()
        
class ConcreteObserverA(AbstractObserver):
    def update(self, state):        
        print(f"> ConcreteObserverA was notified of Subject's new state: {state}")
    
class ConcreteObserverB(AbstractObserver):
    def update(self, state):
        print(f"> ConcreteObserverB was notified of Subject's new state: {state}")
            
subject = ConcreteSubject()

# Register new observers
observerA = ConcreteObserverA()
subject.register(observerA)
observerB = ConcreteObserverB()
subject.register(observerB)

subject.doBusinessLogic()

print("---")

subject.unregister(observerA)
subject.doBusinessLogic()

> ConcreteObserverA was notified of Subject's new state: 7
> ConcreteObserverB was notified of Subject's new state: 7
---
> ConcreteObserverB was notified of Subject's new state: 7


## Factory Method Pattern

The Factory Method Pattern defines an interface for creating an object, but lets subclasses decide which class to instantiate. Factory Method lets a class defer instantiation to subclasses.

![](uml-diagrams/factory-method-pattern.png)

Note to self: You are abstracting the Factory and Product. The Factory MUST have a method that creates a Product. Whenever you add a new Product subclass (e.g. ProductX), you just have to add a corresponding Factory (e.g. ConcreteCreatorX). And when you want to initialize ProductX, call:

```
Creator c = new ConcreteCreatorX()
Product x = c.factory_method() // x is an instance of ProductX
```

### When to Use the Factory Method Pattern

1. When you don't know which class (implementing a common superclass) you will instantiate until runtime
2. When you don't want to expose the creation logic to the client
3. When you want to localize the logic to instantiate objects

#### Pros of Factory Method Pattern

1. Encapsulate and centralize object creation and class selection
2. Greater flexibility in object creation
    - Easy to extend/replace/add new subclasses (e.g. different database implementations)
    - Easy to change object creation logic (e.g. singleton objects, lazy initialization, etc.)
3. Ability to choose which classes to instantiate at runtime

### Skelly Code

In [8]:
class Creator(ABC):
    @abstractmethod
    def factory_method(self):
        pass
    
    def doBusinessLogic(self):
        product = self.factory_method()
        print("Called from the creator:")
        product.operation()

class ConcreteCreatorA(Creator):
    def factory_method(self):
        return ConcreteProductA()
    
class ConcreteCreatorB(Creator):
    def factory_method(self):
        return ConcreteProductB()
    
class Product(ABC):
    @abstractmethod
    def operation(self):
        pass
    
class ConcreteProductA(ABC):
    def operation(self):
        print("ConcreteProductA is operating...")
    
class ConcreteProductB(ABC):
    def operation(self):
        print("ConcreteProductB is operating...")
    
factory = ConcreteCreatorA()
factory.doBusinessLogic()

print("---")

factory = ConcreteCreatorB()
factory.doBusinessLogic()

print("---")

productA = factoryA.factory_method()
productA.operation()

print("---")

productB = ConcreteCreatorB().factory_method()
productB.operation()

Called from the creator:
ConcreteProductA is operating...
Called from the creator:
ConcreteProductB is operating...
---
ConcreteProductA is operating...
---
ConcreteProductB is operating...
---


## Facade Pattern

The Facade Pattern provides a unified interface to a set of interfaces in a subsystem. Facade defines a higher-level interface that makes the subsystem easier to use.

Basically, it provides **convenient access** to a particular part of the subsystem's functionality. E.g. To convert a video, you would have to rely on `AudioMixer`, `VideoFile`, `CodecFactory`, `BitrateReader`, `OggCompressionCodec`, `MPEG4CompressionCodec` classes. Instead of making your main program work with all these extra classes, you create a facade class `VideoConverter` which encapsulates that functionality and hides it from the rest of the code.

#### Pros of Facade Pattern

1. Easy to use and understand
2. Reduced dependencies on other classes
3. Client decoupled from complex system

#### Cons of Facade Pattern

1. Increased complexity
2. Decreased runtime performance if there are a lot of Facade classes

### Skelly Code

In [9]:
class Facade:
    def __init__(self, subsystem1, subsystem2):
        self.subsystem1 = subsystem1
        self.subsystem2 = subsystem2
        
    def operation(self):
        subsystem1.operation()
        subsystem2.operation()
    
class Subsystem1:
    def operation(self):
        print("I'm in Subsystem1!")
    
class Subsystem2:
    def operation(self):
        print("I'm in Subsystem2!")
    
subsystem1 = Subsystem1()
subsystem2 = Subsystem2()
facade = Facade(subsystem1, subsystem2)
facade.operation()